# Lab 3: Word Embeddings and Language Modelling

Adam Ek

In this lab we'll explore constructing *static* word embeddings (i.e. word2vec) and building language models. We'll also evaluate these systems on intermediate tasks, namely word similarity and identifying "good" and "bad" sentences.

* For this we'll use pytorch. Some basic operations that will be useful can be found here: https://jhui.github.io/2018/02/09/PyTorch-Basic-operations
* In general: we are not interested in getting state-of-the-art performance :) focus on the implementation and not results of your model. For this reason, you can use a subset of the dataset: the first 5000-10 000 sentences or so, on linux/mac: ```head -n 10000 inputfile > outputfile```. 
* If possible, use the MLTGpu, it will make everything faster :)

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim
from torchtext.data import Field, BucketIterator, Iterator, TabularDataset

from IPython import embed
import numpy as np

hardware = 'cuda' if torch.cuda.is_available() else 'cpu'
device = torch.device(hardware)

In [5]:
print (device)

cuda


# Word2Vec embeddings

In this first part we'll construct a word2vec model which will give us *static* word embeddings (that is, they are fixed after training).

After we've trained our model we will evaluate the embeddings obtained on a word similarity task.

## Formatting data


First we need to load some data, you can download the file on canvas under files/03-lab-data/wiki-corpus.txt. The file contains 50 000 sentences randomly selected from the complete wikipedia. Each line in the file contains one sentence. The sentences are whitespace tokenized.

Your first task is to create a dataset suitable for word2vec. That is, we define some ```window_size``` then iterate over all sentences in the dataset, putting the center word in one field and the context words in another (separate the fields with ```tab```).

For example, the sentece "this is a lab" with ```window size = 4``` will be formatted as:
```
center, context
---------------------
this    is a lab
is      this a lab
a       this is lab
lab     this is a
```

this will be our training examples when training the word2vec model.

[3 marks]

In [6]:
import string
import pandas as pd

s = "string. With. Punctuation?"
table = str.maketrans(dict.fromkeys(string.punctuation))  # OR {key: None for key in string.punctuation}
new_s = s.translate(table) 

data_path = 'wiki-corpus.txt'
WINDOW_SIZE = 4
def corpus_reader(data_path):
    with open(data_path, encoding="utf-8") as f:
        lines = f.readlines()
        centerWord_contextWords = []
        exceptions_short_sentences_count = 0
        for lineCount,line in enumerate(lines):
            #print (line)
            table = str.maketrans(dict.fromkeys(string.punctuation)) #start punctuation removal
            #print ("table:",table)
            line = line.translate(table) #remove punctuation
            line = line.split()
            #line = ["this","is","a","lab","with","several","words","altogether","making","a","sentence","that","is","long"]
            #line = ["this","is","a","lab"]
            #print ("line:",line)
            #print ("len of line:",len(line))
            for i,word in enumerate(line):
                context_words=[]
                #print ("center word at:",i,"(",word,")")
                if i==0:
                    start_place = 0
                    end_place = WINDOW_SIZE
                    for ii in range(start_place,end_place):
                        #print ("For word",i,"take context word at:",ii)
                        try:
                            context_words.append(line[ii])
                        except IndexError:
                            #print ("exception occured.\nline is:",line,"\nii is: -----",ii)
                            exceptions_short_sentences_count += 1
                else:
                    start_place = i-int(WINDOW_SIZE/2)
                    end_place = i+int(WINDOW_SIZE/2)
                    #print ("start place:", start_place)
                    #print ("end place:",end_place)
                    j=start_place
                    while j<end_place:
                        #print ("j:",j)
                        if start_place<0:
                            start_place += 1
                            end_place += 1
                            #print ("start place now:",start_place,", end place now:",end_place)
                            j += 1
                            continue
                        elif end_place > len(line):
                            #print ("word at i:",i,", end_place:",end_place)
                            for k in range(start_place-1,end_place-1):
                                #print ("k position:",k)
                                #print ("and lastly for word: (",word,"):",line[k])
                                try:
                                    context_words.append(line[k])
                                except IndexError:
                                    pass
                            break
                        #print ("For word",i,"(",word,")","take context word at:",j,"(",line[j],")")
                        context_words.append(line[j])
                        j += 1
                context_words.remove(word)
                #print ("center word:",word,", context words:",context_words)
                centerWord_contextWords.append([word," ".join(context_words)])
            #if lineCount==1:
            #    print ("line 2 centerWord_contextWords:",centerWord_contextWords)
        print ("- note: number of shorter senteces -- than window size -- encountered:",exceptions_short_sentences_count)
        print ("- done creating center word and its context words for window size:",WINDOW_SIZE,"for all lines (",len(lines),") lines")
        df = pd.DataFrame(centerWord_contextWords, columns=['center','context'])
        df.to_csv(data_path+".csv",index=False, sep='\t')
        print ("- done saving center words/context words into csv file. ")
corpus_reader(data_path)

- note: number of shorter senteces -- than window size -- encountered: 918
- done creating center word and its context words for window size: 4 for all lines ( 50000 ) lines
- done saving center words/context words into csv file. 


In [7]:
pd.read_csv("wiki-corpus.txt.csv",sep='\t')

,center,context
0,Anarchist,historian George Woodcock
1,historian,Anarchist George Woodcock
2,George,Anarchist historian Woodcock
3,Woodcock,historian George reports
4,reports,George Woodcock that
...,...,...
1096333,split,race was into
1096334,into,was split eight
1096335,eight,split into stages
1096336,stages,into eight covering


We sampled 50 000 senteces completely random from the *whole* wikipedia for our training data. Give some reasons why this is good, and why it might be bad. (*note*: We'll have a few questions like these, one or two reasons for and against is sufficient)

[2 marks]

**Answer:**

Sampling 50000 diverse Wikipedia sentences is good because it covers a large variation of sentence types with different words and lengths.

However it may not be ideal as it would be too general and would not be enough for such general task; since a general task requires a very large corpus with several times the magnitude of what is currently being sampled.

### Loading the data

We now need to load the data in an appropriate format for torchtext (https://torchtext.readthedocs.io/en/latest/). We'll use PyText for this and it'll follow the same structure as I showed you in the lecture (remember to lower-case all tokens). Create a function which returns a (bucket)iterator of the training data, and the vocabulary object (```Field```). 

(*hint1*: you can format the data such that the center word always is first, then you only need to use one field)

(*hint2*: the code I showed you during the leture is available in /files/pytorch_tutorial/ on canvas)

[4 marks]

In [8]:
def get_data(dataFilePath):
    whitespacer = lambda x: x.split(' ')

    # "fields" that process the different columns in our CSV files
    TOKENS = Field(tokenize    = whitespacer,
                   lower       = True,
                   batch_first = True) # enforce the (batch, words) structure

    # read the csv files
    train = TabularDataset(path = dataFilePath,
                           format = 'csv',
                           fields = [('center', TOKENS),
                                     ('context', TOKENS)],
                           skip_header       = True,
                           csv_reader_params = {'delimiter':'\t',
                                                'quotechar':'⅞'})
    
    # build vocabularies based on what our csv files contained and create word2id mapping
    TOKENS.build_vocab(train, min_freq=1)


    # create batches from our data, and shuffle them for each epoch
    train_iter = BucketIterator(dataset=train,
                                batch_size        = 8,
                                #sort_key=lambda x: len(x.comment_text), #https://github.com/pytorch/text/issues/474
                                sort_within_batch = False, #changed from True to False
                                shuffle           = True,
                                device            = device
                               )

    return train_iter, TOKENS.vocab
    

In [9]:
dataset, vocab = get_data("wiki-corpus.txt.csv")

We lower-cased all tokens above; give some reasons why this is a good idea, and why it may be harmful to our embeddings.

[2 marks]

**Answer:**
- By lower-casing the tokens we are summing up all instances of each token into one unit. For example if we have three instances of "John" & "john" & "johN", we can sum all of them into one "john" with frequency of 3, thus improving our learning and prediction.
- However it might be harmful if two different meanings are meant by the lower-case and the capitalized version of one word, such as names of companies that may have a dictionary meaning as well and cannot be ditinguished as company names if they are lower-cased.

## Word Embeddings Model

We will implement the CBOW model for constructing word embedding models.

In the CBOW model we try to predict the center word based on the context. That is, we take as input ```n``` context words, encode them as vectors, then combine them by summation. This will give us one embedding. We then use this embedding to predict *which* word in our vocabuary is the most likely center word. 

Implement this model 

[7 marks]

In [10]:
class CBOWModel(nn.Module):
    def __init__(self, num_words, num_dim): #...
        super(CBOWModel, self).__init__()
        self.embeddings = nn.Embedding(num_words, num_dim) #...
        self.linear = nn.Linear(num_dim, num_words) #...
    
    def forward(self, context):
        embedded_context = self.embeddings(context) #...
        embedded_context = self.projection_function(embedded_context)
        output = self.linear(embedded_context)
        return output

    def projection_function(self, xs):
        """
        This function will take as input a tensor of size (B, S, D)
        where B is the batch_size, S the window size, and D the dimensionality of embeddings
        this function should compute the sum over the embedding dimensions of the input, 
        that is, we transform (B, S, D) to (B, 1, D) or (B, D) 
        """
        xs_sum = torch.sum(xs, dim=1)
        return xs_sum

In [11]:
CBOWModel(len(vocab),50).to(device)

CBOWModel(
  (embeddings): Embedding(77632, 50)
  (linear): Linear(in_features=50, out_features=77632, bias=True)
)

Now we need to train the models. First we define which hyperparameters to use. (You can change these, for example when *developing* your model you can use a batch size of 2 and a very low dimensionality (say 10), just to speed things up). When actually training your model *fo real*, you can use a batch size of [8,16,32,64], and embedding dimensionality of [128,256].

In [12]:
# you can change these numbers to suit your needs :)
word_embeddings_hyperparameters = {'epochs':3,
                                   'batch_size':16,
                                   'embedding_size':128,
                                   'learning_rate':0.001,
                                   'embedding_dim':128 #32
                                  }

Train your model. Iterate over the dataset, get outputs from your model, calculate loss and backpropagate.

We mentioned in the lecture that we use Negative Log Likelihood (https://pytorch.org/docs/stable/generated/torch.nn.NLLLoss.html) loss to train Word2Vec model. In this lab we'll take a shortcut when *training* and use Cross Entropy Loss (https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html), basically it combines ```log_softmax``` and ```NLLLoss```. So what your model should output is a *score* for each word in our vocabulary. The ```CrossEntropyLoss``` will then assign probabilities and calculate the negative log likelihood loss.

[3 marks]

In [14]:
# load data
dataset, vocab = get_data("wiki-corpus.txt.csv")

# build model and construct loss/optimizer
cbow_model = CBOWModel(len(vocab), word_embeddings_hyperparameters['embedding_dim'])
cbow_model.to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(cbow_model.parameters(), lr=word_embeddings_hyperparameters['learning_rate'])

# start training loop
total_loss = 0
for epoch in range(word_embeddings_hyperparameters['epochs']):
    epoch_loss = 0
    for i, batch in enumerate(dataset):
        
        context = batch.context
        target_word = batch.center
        ##print ("i:",i)
        ##print ("dataset:",dataset)
        ##print ("context:",context)
        ##print ("target_word:",target_word)
        ###print ("target_word view:",target_word.view(-1))
        '''
        target_word view: tensor([3143,   14,   98,    6,    5,   16, 6770,    4])
        '''


        # send your batch of sentences to the model
        output = cbow_model(context)
        
        # compute the loss, you'll need to reshape the input
        # you can read more about this is the documentation for
        # CrossEntropyLoss
        loss = loss_fn(output, target_word.view(-1) )
        total_loss += loss.item()
        epoch_loss += loss.item()
        
        # print average loss for the epoch
        print("Average loss for the epoch:", epoch_loss/(i+1), " --- ", end='\r') 
        #print(total_loss/(i+1)) 
        
        # compute gradients
        loss.backward()
        ####...
        
        # update parameters
        optimizer.step()
        ####...
        
        # reset gradients
        optimizer.zero_grad()
        ####...
    print('\r')
        

Average loss for the epoch: 8.008197490063182  ---  
Average loss for the epoch: 7.776071236444692  ---  
Average loss for the epoch: 7.620276734381541  ---  


## Evaluating the model

We will evaluate the model on a dataset of word similarities, WordSim353 (http://alfonseca.org/eng/research/wordsim353.html , also avalable in vanvas under files/03-l). The first thing we need to do is read the dataset and translate it to integers. What we'll do is to reuse the ```Field``` that records word indexes (the second output of ```get_data()```) and use it to parse the file.

The wordsim data is structured as follows:

```
word1 word2 score
...
```


The ```Field``` we got from ```read_data()``` has two built-in functions, ```stoi``` which maps a string to an integer and ```itos``` which maps an integer to a string. 

What our datareader needs to do is: 

```
for line in file:
    word1, word2, score = file.split()
    # encode word1 and word2 as integers
    word1_idx = vocab.vocab.stoi[word1]
    word2_idx = vocab.vocab.stoi[word2]
```

when we have the integers for ```word_1``` and ```word2``` we'll compute the similarity between their word embeddings with *cosine simlarity*. We can obtain the embeddings by querying the embedding layer of the model.

We calculate the cosine similarity for each word pair in the dataset, then compute the pearson correlation between the similarities we obtained with the scores given in the dataset. 

[4 marks]

In [15]:
print (len(vocab))
print (vocab.stoi["good"])
print (vocab.itos[626])
print (torch.LongTensor([626]))
print (torch.LongTensor())

77632
626
good
tensor([626])
tensor([], dtype=torch.int64)


In [16]:
# your code goes here

def read_wordsim(path, vocab, embeddings):
    word_pairs = []
    dataset_sims = []
    model_sims = []
    with open(path) as f:
        for line in f:
            word1, word2, score = line.split()
            word_pairs.append((word1,word2))
            
            score = float(score)
            dataset_sims.append(score)
            
                # get the index for the word
            word1_idx = vocab.stoi[word1.lower()] #...
            word2_idx = vocab.stoi[word2.lower()] #...
            
            # get the embedding of the word
            word1_emb = embeddings(torch.LongTensor([word1_idx]).to(device)) #...
            word2_emb = embeddings(torch.LongTensor([word2_idx]).to(device)) #...

            #print ("word:", word1, "--",embeddings(torch.LongTensor([word1_idx]).to(device)))

            # compute cosine similarity, we'll use the version included in pytorch functional
            # https://pytorch.org/docs/master/generated/torch.nn.functional.cosine_similarity.html
            cosine_similarity = F.cosine_similarity(word1_emb,word2_emb)
            #print (cosine_similarity.item())
            
            model_sims.append(cosine_similarity.item())
    
    return dataset_sims, model_sims, word_pairs

path = 'wordsim_similarity_goldstandard.txt'
data, model, pairs = read_wordsim(path,vocab,cbow_model.embeddings)
#print (data)
#print (model)
pearson_correlation = np.corrcoef(data, model)
            
# the non-diagonals give the pearson correlation,
print(pearson_correlation)

[[1.         0.23241196]
 [0.23241196 1.        ]]


Do you think the model performs good or bad? Why?

[3 marks]

**Answer**
The model is performing poorly when the pearson correlation coefficient is closer to 0 than being closer to 1.
The result from the model is a low number and thus we can reason that the model is performing weakly.

Select the 10 best and 10 worst performing word pairs, can you see any patterns that explain why *these* are the best and worst word pairs?

[3 marks]

**Answer**
The code below shows 10 best and 10 worst performing word pairs.
From the best performing words, it can be seen that words that are in the same category
or subject area have most similarity where as words that may have multiple meanings such
as "problem" or words that are very irrelevant to each other in their pair make for worst
performing pairs, such as "cucumber" and "professor", or "cabage" and "king".

In [17]:
workingData = data[:]; workingModel = model[:]; workingPairs = pairs[:];
best10 = []
for i in range(0,len(workingPairs)):
    bestIndex = workingModel.index(max(workingModel))
    #print (workingData[bestIndex])
    maxTuple = (workingData[bestIndex],workingModel[bestIndex], workingPairs[bestIndex])
    best10.append(maxTuple)
    del workingData[bestIndex]; del workingModel[bestIndex]; del workingPairs[bestIndex];
    if (i==9):
        break
print ("Best 10 performing word pairs:")
for i in range (0,len(best10)):
    print (best10[i])
#print (best10)

workingData = data[:]; workingModel = model[:]; workingPairs = pairs[:];
worst10 = []
for i in range(0,len(workingPairs)):
    worstIndex = workingModel.index(min(workingModel))
    minTuple = (workingData[worstIndex],workingModel[worstIndex], workingPairs[worstIndex])
    worst10.append(minTuple)
    del workingData[worstIndex]; del workingModel[worstIndex]; del workingPairs[worstIndex];
    if (i==9):
        break
print ("Worst 10 performing word pairs:")
for i in range (0,len(worst10)):
    print (worst10[i])

Best 10 performing word pairs:
(10.0, 1.0, ('tiger', 'tiger'))
(8.3, 0.5581941604614258, ('man', 'woman'))
(6.77, 0.4818370044231415, ('television', 'radio'))
(8.97, 0.45765310525894165, ('type', 'kind'))
(6.81, 0.4164801836013794, ('football', 'basketball'))
(9.1, 0.35958829522132874, ('coast', 'shore'))
(8.02, 0.35037505626678467, ('planet', 'sun'))
(4.81, 0.34970614314079285, ('situation', 'conclusion'))
(5.77, 0.331089049577713, ('plane', 'car'))
(6.22, 0.3248131275177002, ('skin', 'eye'))
Worst 10 performing word pairs:
(1.19, -0.2852497100830078, ('delay', 'racism'))
(7.81, -0.19720673561096191, ('lobster', 'food'))
(2.94, -0.16753925383090973, ('peace', 'insurance'))
(8.0, -0.1426965892314911, ('tiger', 'feline'))
(8.42, -0.13960173726081848, ('money', 'dollar'))
(4.25, -0.12836161255836487, ('attempt', 'peace'))
(2.31, -0.11290749162435532, ('reason', 'hypertension'))
(2.38, -0.10981321334838867, ('problem', 'airport'))
(1.92, -0.10487410426139832, ('cup', 'substance'))
(3.78, 

Suggest some ways of improving the model we apply to WordSim353.

[3 marks]

**Answer**
To improve the results, training on more sentences can be done. More context for words
that are similar but interpreted as different should be given to the model.

If we consider a scenario where we use these embeddings in a downstream task, for example sentiment analysis (roughly: determining whether a sentence is positive or negative). 

Give some examples why the sentiment analysis model would benefit from our embeddnings and one examples why our embeddings could hur the performance of the sentiment model.

[3 marks]

**Answer**
Sentiment analysis would benefit from the similarity of words used to
explain a positive or negative topic. In these cases, words such as "cool", "great",
and "like" would lead to the sentence falling into the same sentiment category.

Where our embedding can hurt the performance is use of short negative words that
would be the only cues as a negative or positive expression, such as the word
"not" in "I did not like this 'awesome' product!". In such case, only one word makes
the sentence meaning change and it may be difficult for the model to spot such word
and make the right conclusion about the sentence polarity.
Words may also have different meanings or used in ironies to refer to a different meaning
and purpose, which may not be spotted by our embeddings, such as the example we see above
with the words "cabbage" and "king", i.e. use of irony, interpreted as irrelevant and
dissimilar.

# Language modeling

In this second part we'll build a simple LSTM language model. Your task is to construct a model which takes a sentence as input and predict the next word for each word in the sentence. For this you'll use the ```LSTM``` class provided by PyTorch (https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html). You can read more about the LSTM here: https://colah.github.io/posts/2015-08-Understanding-LSTMs/

NOTE!!!: Use the same dataset (wiki-cropus.txt) as before.

Our setup is similar to before, we first encode the words as distributed representations then pass these to the LSTM and for each output we predict the next word.

For this we'll build a new dataloader with torchtext, the file we pass to the dataloader should contain one sentence per line, with words separated by whitespace.

```
word_1, ..., word_n
word_1, ..., word_k
...
```

in this dataloader you want to make sure that each sentence begins with a ```<start>``` token and ends with a ```<end>``` token, there is a keyword argument in ```Field``` for this :). But other than that, as before you read the dataset and output a iterator over the dataset and a vocabulary. 

Implement the dataloader, language model and the training loop (the training loop will basically be the same as for word2vec).

[12 marks]

In [20]:
CUDA_LAUNCH_BLOCKING=1

In [21]:
# you can change these numbers to suit your needs as before :)
lm_hyperparameters = {'epochs':3,
                      'batch_size':16,
                      'learning_rate':0.001,
                      'embedding_dim':128,
                      'output_dim':128}

In [22]:
data_path = 'wiki-corpus.txt'
def get_data_language_model(dataFilePath):
    # your code here, roughly the same as for the word2vec dataloader
    whitespacer = lambda x: x.split(' ')

    # "fields" that process the different columns in our CSV files
    sentence = Field(tokenize    = whitespacer,
                   init_token = '<start>',
                   eos_token = '<end>',
                   lower       = True,
                   batch_first = True) # enforce the (batch, words) structure

    # read the csv files
    train = TabularDataset(path = dataFilePath,
                           format = 'csv',
                           fields = [('sentence', sentence),
                                     ],
                           )
    
    # build vocabularies based on what our csv files contained and create word2id mapping
    sentence.build_vocab(train)


    # create batches from our data, and shuffle them for each epoch
    train_iter = BucketIterator(dataset=train,
                                batch_size        = 8,
                                #sort_key=lambda x: len(x.comment_text), #https://github.com/pytorch/text/issues/474
                                sort_within_batch = False, #changed from True to False
                                shuffle           = True,
                                device            = device)

    return train_iter, sentence.vocab


In [23]:
dataset_language_model, vocab_language_model = get_data_language_model(data_path)

In [24]:
print (vocab_language_model)
print (vocab_language_model.stoi["<end>"])
print (len(vocab_language_model))

#src_map = torch.LongTensor([vocab_language_model.stoi[w] for w in src])
print (vocab_language_model.stoi["is"])
print ("dataset_language_model:",dataset_language_model)

3
50628
12
dataset_language_model: <torchtext.data.iterator.BucketIterator object at 0x7fba4c284710>


In [25]:
class LM_withLSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, n_layers=1):
        super(LM_withLSTM, self).__init__()
        self.n_layers = n_layers
        
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.LSTM = nn.LSTM(embedding_dim, hidden_dim, n_layers, batch_first=True)
        self.predict_word = nn.Linear(hidden_dim, vocab_size)
        #self.sigmoid = nn.Sigmoid()
    
    def forward(self, seq):
        embedded_seq = self.embeddings(seq)
        timestep_reprentation, *_ = self.LSTM(embedded_seq)
        predicted_words = self.predict_word(timestep_reprentation)
        
        return predicted_words

In [26]:
# load data
#lm_dataset, lm_vocab = get_data_language_model(data_path)

# build model and construct loss/optimizer
lm_model = LM_withLSTM(len(vocab_language_model), 
                       lm_hyperparameters['embedding_dim'],
                       lm_hyperparameters['output_dim'])
lm_model.to(device)
#lm_model.eval()


LM_withLSTM(
  (embeddings): Embedding(50628, 128)
  (LSTM): LSTM(128, 128, batch_first=True)
  (predict_word): Linear(in_features=128, out_features=50628, bias=True)
)

In [27]:

loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(lm_model.parameters(), lr=lm_hyperparameters['learning_rate'])

# start training loop
total_loss = 0
for epoch in range(lm_hyperparameters['epochs']):
    for i, batch in enumerate(dataset_language_model):
        #print ("i:",i)
        # the strucure for each BATCH is:
        # <start>, w0, ..., wn, <end>
        sentence = batch.sentence
        
        # when training the model, at each input we predict the *NEXT* token
        # consequently there is nothing to predict when we give the model 
        # <end> as input. 
        # thus, we do not want to give <end> as input to the model, select 
        # from each batch all tokens except the last. 
        # tip: use pytorch indexing/slicing (same as numpy) 
        # (https://pytorch.org/tutorials/beginner/basics/tensorqs_tutorial.html#operations-on-tensors)
        # (https://jhui.github.io/2018/02/09/PyTorch-Basic-operations/)
        input_sentence = sentence[0:,range(sentence.shape[1]-1)] #...
        
        # send your batch of sentences to the model
        output = lm_model(input_sentence)
        
        # for each output, the model predict the NEXT token, so we have to reshape 
        # our dataset again. On timestep t, we evaluate on token t+1. That is,
        # we never predict the <start> token ;) so this time, we select all but the first 
        # token from sentences (that is, all the tokens that we predict)
        gold_data = sentence[0:,range(1,sentence.shape[1])]
        
        # the shape of the output and sentence variable need to be changed,
        # for the loss function. Details are in the documentation.
        # You can use .view(...,...) to reshape the tensors  
        loss = loss_fn(output.view(-1,len(vocab_language_model)),  gold_data.view(-1)) #...check
        total_loss += loss.item()
        
        # print average loss for the epoch
        print(total_loss/(i+1), end='\r') 
        
        # compute gradients
        loss.backward() #...
        
        # update parameters
        optimizer.step() ####...
        
        # reset gradients
        optimizer.zero_grad() #...
    print()

3.1200737282168795
5.8188218711795655
8.3210848349917513


### Evaluating the language model

We'll evaluate our model using the BLiMP dataset (https://github.com/alexwarstadt/blimp). The BLiMP dataset contains sets of linguistic minimal pairs for various syntactic and semantic phenomena, We'll evaluate our model on *existential quantifiers* (link: https://github.com/alexwarstadt/blimp/blob/master/data/existential_there_quantifiers_1.jsonl). This data, as the name suggests, investigate whether language models assign higher probability to *correct* usage of there-quantifiers. 

An example entry in the dataset is: 

```
{"sentence_good": "There was a documentary about music irritating Allison.", "sentence_bad": "There was each documentary about music irritating Allison.", "field": "semantics", "linguistics_term": "quantifiers", "UID": "existential_there_quantifiers_1", "simple_LM_method": true, "one_prefix_method": false, "two_prefix_method": false, "lexically_identical": false, "pairID": "0"}
```

Download the dataset and build a datareader (similar to what you did for word2vec). The dataset structure you should aim for is (you don't need to worry about the other keys for this assignment):

```
good_sentence_1, bad_sentence_1
...
```

your task now is to compare the probability assigned to the good sentence with to the probability assigned to the bad sentence. To compute a probability for a sentence we consider the product of the probabilities assigned to the *gold* tokens, remember, at timestep ```t``` we're predicting which token comes *next* e.g. ```t+1``` (basically, you do the same thing as you did when training).

In rough pseudo code what your code should do is:

```
accuracy = []
for good_sentence, bad_sentence in dataset:
    gs_lm_output = LanguageModel(good_sentence)
    gs_token_probabilities = softmax(gs_lm_output)
    gs_sentence_probability = product(gs_token_probabilities[GOLD_TOKENS])

    bs_lm_output = LanguageModel(bad_sentence)
    bs_token_probabilities = softmax(bs_lm_output)
    bs_sentence_probability = product(bs_token_probabilities[GOLD_TOKENS])

    # int(True) = 1 and int(False) = 0
    is_correct = int(gs_sentence_probability > bs_sentence_probability)
    accuracy.append(is_correct)

print(numpy.mean(accuracy))
    
```

[6 marks]

In [28]:
# your code goes here
import json

def evaluate_model(path, vocab, model):
    
    accuracy = []
    with open(path) as f:
        # iterate over one pair of sentences at a time
        for line in f:
            # load the data
            data = json.loads(line)
            good_s = data['sentence_good']
            print ("good s:",good_s)
            bad_s = data['sentence_bad']
            
            # the data is tokenized as whitespace
            tok_good_s = [vocab.stoi[word] for word in ['<start>']+good_s.lower().rstrip('.').split()+['.','<end>']]
            print ("tok_good_s:", tok_good_s)
            
            tok_bad_s = [vocab.stoi[word] for word in ['<start>']+bad_s.lower().rstrip('.').split()+['.','<end>']]
            
            # encode your words as integers using the vocab from the dataloader, size is (S)
            # we use unsqueeze to create the batch dimension 
            # in this case our input is only ONE batch, so the size of the tensor becomes: 
            # (S) -> (1, S) as the model expects batches
            enc_good_s = torch.tensor([x for x in tok_good_s], device=device).unsqueeze(0)
            print("enc_good_s:",enc_good_s)
            enc_bad_s = torch.tensor([x for x in tok_bad_s], device=device).unsqueeze(0)
            
            # pass your encoded sentences to the model and predict the next tokens
            good_s = model(enc_good_s)
            print("good_s:",good_s)
            bad_s = model(enc_bad_s)
            
            # get probabilities with softmax
            gs_probs = F.softmax(good_s, dim=1)
            print ("gs_probs:",gs_probs)            
            bs_probs = F.softmax(bad_s, dim=1)
            
            # select the probability of the gold tokens
            gs_sent_prob = find_token_probs(gs_probs, tok_good_s)
            bs_sent_prob = find_token_probs(bs_probs, tok_bad_s)
            
            accuracy.append(int(gs_sent_prob>bs_sent_prob))

    return accuracy
            
def find_token_probs(model_probs, encoded_sentence):
    probs = []

    # iterate over the tokens in your encoded sentence
    print ("## find_token_probs...##")
    for sent_idx,vocab_idx in [(i-1,t) for i,t in enumerate(encoded_sentence)][1:]:
        # select the probability of the gold tokens and save
        # hint: pytorch indexing is helpful here ;)
        
        prob = model_probs.view(-1,len(vocab_language_model))[sent_idx][vocab_idx]  
        print ("prob:",prob)
        probs.append(prob)
    sentence_prob = np.prod(probs)

    #print ("encoded_sentence[0]:",encoded_sentence[0])
    return sentence_prob

path = 'existential_there_quantifiers_1.jsonl'



In [29]:
###accuracy = evaluate_model(path, ..., ...)
accuracy = evaluate_model(path, vocab_language_model, lm_model)

good s: There was a documentary about music irritating Allison.
tok_good_s: [2, 40, 13, 9, 1952, 98, 216, 0, 0, 8, 3]
enc_good_s: tensor([[   2,   40,   13,    9, 1952,   98,  216,    0,    0,    8,    3]],
       device='cuda:0')
good_s: tensor([[[-11.3195,  -4.5230, -11.3360,  ...,  -6.6381,  -7.1910,  -6.7362],
         [-17.1620,  -3.1561, -17.5434,  ..., -17.0393, -18.6054, -19.4060],
         [-11.5907,  -3.7124, -11.7903,  ...,  -7.4221,  -7.5123,  -8.3519],
         ...,
         [-11.2607,  -5.8681, -11.1289,  ...,  -6.9956, -10.4502, -10.2464],
         [-20.1039,  -4.9261, -20.6242,  ...,  -8.1029, -11.1276, -13.3413],
         [-15.6879,  19.1810, -16.5931,  ..., -10.4611, -12.8756, -17.0511]]],
       device='cuda:0', grad_fn=<AddBackward0>)
gs_probs: tensor([[[1.0877e-01, 5.0757e-11, 1.0593e-01,  ..., 1.2824e-01,
          4.1469e-01, 6.4562e-01],
         [3.1561e-04, 1.9912e-10, 2.1341e-04,  ..., 3.8979e-06,
          4.5762e-06, 2.0304e-06],
         [8.2931e-02, 1.141

prob: tensor(0.0011, device='cuda:0', grad_fn=<SelectBackward>)
prob: tensor(0.0615, device='cuda:0', grad_fn=<SelectBackward>)
prob: tensor(0.9997, device='cuda:0', grad_fn=<SelectBackward>)
good s: There is a mother of Theodore imagining those adults responded.
tok_good_s: [2, 40, 12, 9, 1084, 6, 6974, 0, 285, 3927, 4108, 8, 3]
enc_good_s: tensor([[   2,   40,   12,    9, 1084,    6, 6974,    0,  285, 3927, 4108,    8,
            3]], device='cuda:0')
good_s: tensor([[[-11.3195,  -4.5230, -11.3360,  ...,  -6.6381,  -7.1910,  -6.7362],
         [-17.1620,  -3.1561, -17.5434,  ..., -17.0393, -18.6054, -19.4060],
         [-11.3934,  -1.0657, -11.6936,  ...,  -8.8110, -10.4502, -10.1988],
         ...,
         [-13.7326,  -3.1549, -14.0650,  ...,  -6.3935, -12.4656, -10.2735],
         [-20.1678,  -5.5758, -20.5983,  ...,  -9.8300,  -9.3879, -12.7238],
         [-16.6251,  19.0900, -17.5935,  ..., -11.0362, -13.0193, -16.8268]]],
       device='cuda:0', grad_fn=<AddBackward0>)
gs_prob

prob: tensor(0.0020, device='cuda:0', grad_fn=<SelectBackward>)
prob: tensor(0.0163, device='cuda:0', grad_fn=<SelectBackward>)
prob: tensor(0.3561, device='cuda:0', grad_fn=<SelectBackward>)
prob: tensor(0.1439, device='cuda:0', grad_fn=<SelectBackward>)
prob: tensor(0.9994, device='cuda:0', grad_fn=<SelectBackward>)
## find_token_probs...##
prob: tensor(0.7340, device='cuda:0', grad_fn=<SelectBackward>)
prob: tensor(0.0003, device='cuda:0', grad_fn=<SelectBackward>)
prob: tensor(0.1578, device='cuda:0', grad_fn=<SelectBackward>)
prob: tensor(0.0066, device='cuda:0', grad_fn=<SelectBackward>)
prob: tensor(0.0015, device='cuda:0', grad_fn=<SelectBackward>)
prob: tensor(0.0359, device='cuda:0', grad_fn=<SelectBackward>)
prob: tensor(0.1466, device='cuda:0', grad_fn=<SelectBackward>)
prob: tensor(0.1846, device='cuda:0', grad_fn=<SelectBackward>)
prob: tensor(0.9992, device='cuda:0', grad_fn=<SelectBackward>)
good s: There are no peppers upsetting all governments.
tok_good_s: [2, 40, 27,

enc_good_s: tensor([[    2,    40,    22,    87, 10979, 27079,  9073,     8,     3]],
       device='cuda:0')
good_s: tensor([[[-11.3195,  -4.5230, -11.3360,  ...,  -6.6381,  -7.1910,  -6.7362],
         [-17.1620,  -3.1561, -17.5434,  ..., -17.0393, -18.6054, -19.4060],
         [ -7.0473,  -1.7611,  -7.0126,  ...,  -6.3974,  -3.2409,  -5.4860],
         ...,
         [-12.3937,  -9.3396, -12.5298,  ...,  -5.5441,  -8.8380,  -6.6118],
         [-20.6631,  -6.2810, -21.2443,  ...,  -7.9873, -11.8261, -12.4810],
         [-16.6499,  18.6079, -17.6265,  ..., -11.1206, -11.8696, -15.7193]]],
       device='cuda:0', grad_fn=<AddBackward0>)
gs_probs: tensor([[[1.3685e-02, 9.0033e-11, 1.3024e-02,  ..., 1.7949e-01,
          1.8804e-02, 1.7709e-01],
         [3.9709e-05, 3.5320e-10, 2.6237e-05,  ..., 5.4556e-06,
          2.0750e-07, 5.5691e-07],
         [9.8095e-01, 1.4252e-09, 9.8263e-01,  ..., 2.2834e-01,
          9.7667e-01, 6.1821e-01],
         ...,
         [4.6744e-03, 7.2871e-13, 3

prob: tensor(0.7024, device='cuda:0', grad_fn=<SelectBackward>)
prob: tensor(0.9966, device='cuda:0', grad_fn=<SelectBackward>)
prob: tensor(0.9761, device='cuda:0', grad_fn=<SelectBackward>)
prob: tensor(0.1303, device='cuda:0', grad_fn=<SelectBackward>)
prob: tensor(0.0140, device='cuda:0', grad_fn=<SelectBackward>)
prob: tensor(0.0313, device='cuda:0', grad_fn=<SelectBackward>)
prob: tensor(0.2159, device='cuda:0', grad_fn=<SelectBackward>)
prob: tensor(0.4255, device='cuda:0', grad_fn=<SelectBackward>)
prob: tensor(0.9997, device='cuda:0', grad_fn=<SelectBackward>)
## find_token_probs...##
prob: tensor(0.7114, device='cuda:0', grad_fn=<SelectBackward>)
prob: tensor(0.9948, device='cuda:0', grad_fn=<SelectBackward>)
prob: tensor(0.2661, device='cuda:0', grad_fn=<SelectBackward>)
prob: tensor(0.0012, device='cuda:0', grad_fn=<SelectBackward>)
prob: tensor(0.2396, device='cuda:0', grad_fn=<SelectBackward>)
prob: tensor(0.0237, device='cuda:0', grad_fn=<SelectBackward>)
prob: tensor(0.

...

gs_probs: tensor([[[3.2782e-02, 1.3494e-10, 2.4521e-02,  ..., 2.0311e-02,
          1.7371e-01, 3.7818e-01],
         [9.5119e-05, 5.2935e-10, 4.9398e-05,  ..., 6.1737e-07,
          1.9170e-06, 1.1893e-06],
         [3.0447e-02, 4.2814e-09, 1.7149e-02,  ..., 2.3124e-03,
          6.6740e-03, 1.1856e-02],
         ...,
         [1.2831e-03, 3.9878e-11, 6.9532e-04,  ..., 1.4616e-02,
          6.0478e-02, 4.8993e-02],
         [2.1877e-05, 8.4198e-11, 1.0254e-05,  ..., 5.8777e-03,
          2.2344e-01, 3.8286e-03],
         [8.4904e-04, 1.0000e+00, 3.0503e-04,  ..., 1.0619e-03,
          1.3183e-03, 3.2554e-05]]], device='cuda:0',
       grad_fn=<SoftmaxBackward>)
## find_token_probs...##
prob: tensor(0.7417, device='cuda:0', grad_fn=<SelectBackward>)
prob: tensor(0.9794, device='cuda:0', grad_fn=<SelectBackward>)
prob: tensor(0.7615, device='cuda:0', grad_fn=<SelectBackward>)
prob: tensor(0.2886, device='cuda:0', grad_fn=<SelectBackward>)
prob: tensor(3.0345e-05, device='cuda:0', grad_f

prob: tensor(0.7075, device='cuda:0', grad_fn=<SelectBackward>)
prob: tensor(0.0001, device='cuda:0', grad_fn=<SelectBackward>)
prob: tensor(0.1186, device='cuda:0', grad_fn=<SelectBackward>)
prob: tensor(0.3541, device='cuda:0', grad_fn=<SelectBackward>)
prob: tensor(0.0024, device='cuda:0', grad_fn=<SelectBackward>)
prob: tensor(0.0280, device='cuda:0', grad_fn=<SelectBackward>)
prob: tensor(0.4389, device='cuda:0', grad_fn=<SelectBackward>)
prob: tensor(0.9909, device='cuda:0', grad_fn=<SelectBackward>)
prob: tensor(0.1138, device='cuda:0', grad_fn=<SelectBackward>)
prob: tensor(0.9977, device='cuda:0', grad_fn=<SelectBackward>)
## find_token_probs...##
prob: tensor(0.6768, device='cuda:0', grad_fn=<SelectBackward>)
prob: tensor(0.0003, device='cuda:0', grad_fn=<SelectBackward>)
prob: tensor(0.1418, device='cuda:0', grad_fn=<SelectBackward>)
prob: tensor(0.0718, device='cuda:0', grad_fn=<SelectBackward>)
prob: tensor(0.0048, device='cuda:0', grad_fn=<SelectBackward>)
prob: tensor(0.

good_s: tensor([[[-11.3195,  -4.5230, -11.3360,  ...,  -6.6381,  -7.1910,  -6.7362],
         [-17.1620,  -3.1561, -17.5434,  ..., -17.0393, -18.6054, -19.4060],
         [-11.3934,  -1.0657, -11.6936,  ...,  -8.8110, -10.4502, -10.1988],
         ...,
         [-12.7809,  -6.3241, -12.4531,  ...,  -5.6389, -16.2976, -12.5366],
         [-20.2338,  -4.0442, -20.7443,  ...,  -8.8463, -13.0953, -14.1525],
         [-17.2736,  19.5952, -18.2961,  ..., -11.6346, -13.4069, -17.5427]]],
       device='cuda:0', grad_fn=<AddBackward0>)
gs_probs: tensor([[[3.0257e-01, 3.3543e-11, 3.0371e-01,  ..., 1.8968e-01,
          8.8016e-01, 9.3061e-01],
         [8.7793e-04, 1.3159e-10, 6.1182e-04,  ..., 5.7654e-06,
          9.7128e-06, 2.9266e-06],
         [2.8102e-01, 1.0643e-09, 2.1240e-01,  ..., 2.1594e-02,
          3.3815e-02, 2.9175e-02],
         ...,
         [7.0171e-02, 5.5380e-12, 9.9388e-02,  ..., 5.1516e-01,
          9.7636e-05, 2.8164e-03],
         [4.0680e-05, 5.4141e-11, 2.4918e-05, 

prob: tensor(0.0454, device='cuda:0', grad_fn=<SelectBackward>)
prob: tensor(0.4192, device='cuda:0', grad_fn=<SelectBackward>)
prob: tensor(0.9992, device='cuda:0', grad_fn=<SelectBackward>)
## find_token_probs...##
prob: tensor(0.7061, device='cuda:0', grad_fn=<SelectBackward>)
prob: tensor(0.9871, device='cuda:0', grad_fn=<SelectBackward>)
prob: tensor(0.1026, device='cuda:0', grad_fn=<SelectBackward>)
prob: tensor(0.0110, device='cuda:0', grad_fn=<SelectBackward>)
prob: tensor(0.0292, device='cuda:0', grad_fn=<SelectBackward>)
prob: tensor(0.9054, device='cuda:0', grad_fn=<SelectBackward>)
prob: tensor(0.9688, device='cuda:0', grad_fn=<SelectBackward>)
prob: tensor(0.0288, device='cuda:0', grad_fn=<SelectBackward>)
prob: tensor(0.4609, device='cuda:0', grad_fn=<SelectBackward>)
prob: tensor(0.9993, device='cuda:0', grad_fn=<SelectBackward>)
good s: There were some newspaper articles about literature looking like the print.
tok_good_s: [2, 40, 22, 57, 1855, 1730, 98, 680, 3212, 313,

good s: There wasn't a child dropping by some malls.
tok_good_s: [2, 40, 0, 9, 912, 12457, 16, 57, 13263, 8, 3]
enc_good_s: tensor([[    2,    40,     0,     9,   912, 12457,    16,    57, 13263,     8,
             3]], device='cuda:0')
good_s: tensor([[[-11.3195,  -4.5230, -11.3360,  ...,  -6.6381,  -7.1910,  -6.7362],
         [-17.1620,  -3.1561, -17.5434,  ..., -17.0393, -18.6054, -19.4060],
         [ -9.3842,  -2.9625,  -9.4576,  ...,  -6.8620,  -7.3142,  -8.7988],
         ...,
         [-10.2695,  -4.2788, -10.3018,  ...,  -3.9746,  -4.7006,  -5.0540],
         [-19.4901,  -5.6295, -20.0619,  ...,  -7.5964,  -8.6155, -11.3603],
         [-16.5302,  19.2424, -17.5076,  ..., -11.3882, -13.2870, -16.9622]]],
       device='cuda:0', grad_fn=<AddBackward0>)
gs_probs: tensor([[[3.7451e-02, 4.7735e-11, 4.0524e-02,  ..., 4.1704e-02,
          3.5384e-02, 1.4429e-01],
         [1.0867e-04, 1.8727e-10, 8.1635e-05,  ..., 1.2676e-06,
          3.9047e-07, 4.5377e-07],
         [2.5938e-01

In [30]:
print('Final accuracy:')
print(np.round(np.mean(accuracy), 3))

Final accuracy:
0.818


### Analysis

Our model get some score, say, 55% correct predictions. Is this good? Suggest some *baseline* (i.e. a stupid "model" we hope ours is better than) we can compare the model against.

[3 marks]

**Answer**
Our model is performing good by giving higher probablity to good sentneces,
compared with bas sentences. A baseline model would be one that does not use the LSTM layer
and predicts next words given only one previous word.

Suggest some improvements you could make to your language model.

[3 marks]

**Answer**
A larger dataset and batch size can improve the the result from the language model. Also if the model would break down words into characters and predict based on word characters it would have better word predictions for new unknown words that may fall in the same grammatical category

Suggest some other metrics we can use to evaluate our system

[2 marks]

**Answer**
Other gold datasets that can be utilized are datasets that deal with correct word orders in advanced language use. Such word orders would contribute to better sentence probability outcomes compared with daily life sentences that may not have considered grammatically correct word orders.

# Literature


Neural architectures:
* Y. Bengio, R. Ducharme, P. Vincent, and C. Janvin. A neural probabilistic language model. (Links to an external site.) Journal of Machine Learning Research, 3(6):1137–1155, 2003. (Sections 3 and 4 are less relevant today and hence you can glance through them quickly. Instead, look at the Mikolov papers where they describe training word embeddings with the current neural network architectures.)
* T. Mikolov, K. Chen, G. Corrado, and J. Dean. Efficient estimation of word representations in vector space. arXiv preprint arXiv:1301.3781, 2013.
* T. Mikolov, I. Sutskever, K. Chen, G. S. Corrado, and J. Dean. Distributed representations of words and phrases and their compositionality. In Advances in neural information processing systems, pages 3111–3119, 2013.
    


Total marks: 63